In [2]:
import logging,sys
log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

In [3]:
    import os
    assert 'THEANO_FLAGS' in os.environ
    # in case you want to switch to cpu, to be able to use more than one notebook
    os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True,optimizer=None'

In [4]:
from convvisual.mnist import *
from convvisual.receptive_field.convvisual import *
from convvisual.receptive_field.layers import DilationSeparate2DLayer,DilationMerge2DLayer
import lasagne
import matplotlib.pyplot as plt
%matplotlib inline
from numpy.random import RandomState
import convvisual.tests.test_dilation
import convvisual.tests.test_inverse_conv

%load_ext autoreload
%autoreload 2

In [14]:
rng = RandomState(5)

n_layers = rng.randint(8)
shape_input = rng.randint(100,500,size=(2,1))
filters = rng.randint(1,4,size=(n_layers,1))
filter_size = rng.randint(1,5,size=(n_layers,2))
pools = rng.randint(1,4,size=(n_layers,2))
strides = rng.randint(1,3,size=(n_layers,2))

l_in = lasagne.layers.InputLayer((None,1,shape_input[0,0],shape_input[1,0]))
l_last = l_in
for i in np.arange(n_layers):
    l_next = lasagne.layers.Conv2DLayer(l_last,
                            num_filters=filters[i,0],
                            filter_size=list(filter_size[i,:]),
                            nonlinearity=lasagne.nonlinearities.elu)
    l_last = lasagne.layers.Pool2DLayer(l_next,
                                    pool_size=list(pools[i,:]),
                                    stride=list(strides[i,:]))
    
l_out = lasagne.layers.Conv2DLayer(l_last,
                                   num_filters=4,
                                   filter_size=(2,4))


In [15]:
from braindecode.veganlasagne.layer_util import print_layers
layers = lasagne.layers.get_all_layers(l_out)
print_layers(layers)

 0-InputLayer                                                         (None, 1, 306, 289)
 1-Conv2DLayer              2x1                       elu             (None, 3, 305, 289)
 2-Pool2DLayer              1x3 ::2 ::1               max             (None, 3, 153, 287)
 3-Conv2DLayer              1x4                       elu             (None, 3, 153, 284)
 4-Pool2DLayer              1x1 ::2 ::1               max             (None, 3, 77, 284)
 5-Conv2DLayer              3x4                       elu             (None, 1, 75, 281)
 6-Pool2DLayer              2x2 ::2 ::2               max             (None, 1, 37, 140)
 7-Conv2DLayer              2x4                       rectify         (None, 4, 36, 137)



In [16]:
X = rng.randint(100,500,size=(1,1,shape_input[0,0],shape_input[1,0]))
X = np.asarray(X,dtype=np.float32)
output = lasagne.layers.get_output(l_out,X)

/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [17]:
l_deconv = receptive_field_build_deconv_layers(l_out,layers[1])

In [18]:
deconv_layers = lasagne.layers.get_all_layers(l_deconv)
print_layers(deconv_layers)
assert(np.array_equal(deconv_layers[-1].output_shape,layers[0].shape))

 0-InputLayer                                                         (None, 4, 36, 137)
 1-ReversedConv2DLayer      2x4                       rectify         (None, 1, 37, 140)
 2-ReversedPool2DLayer      2x2 ::2 ::2               rectify         (None, 1, 75, 281)
 3-ReversedConv2DLayer      3x4                       rectify         (None, 3, 77, 284)
 4-ReversedPool2DLayer      1x1 ::2 ::1               rectify         (None, 3, 153, 284)
 5-ReversedConv2DLayer      1x4                       rectify         (None, 3, 153, 287)
 6-ReversedPool2DLayer      1x3 ::2 ::1               rectify         (None, 3, 305, 289)
 7-ReversedConv2DLayer      2x1                       rectify         (None, 1, 306, 289)



In [19]:
l_in = lasagne.layers.InputLayer((None,3,8,8))
l_c1 = lasagne.layers.Conv2DLayer(l_in,
                            num_filters=3,
                            filter_size=(3,3),
                            stride=(2,2),
                            pad='full',
                            W=lasagne.init.Constant(1),
                            nonlinearity=lasagne.nonlinearities.elu)
l_p1 = lasagne.layers.Pool2DLayer(l_c1,
                                pool_size=(3,3),
                                stride=(2,2),
                                ignore_border=False)
l_shuffle = lasagne.layers.DimshuffleLayer(l_p1,(0,1,3,2))
l_r1 = lasagne.layers.ReshapeLayer(l_shuffle,([0],1,1,-1))
#l_b1 = lasagne.layers.BatchNormLayer(l_r1)
#l_d1 = lasagne.layers.DropoutLayer(l_b1)
#l_n1 = lasagne.layers.NonlinearityLayer(l_b1)
l_out = l_r1


In [20]:
layers = lasagne.layers.get_all_layers(l_out)
print_layers(layers)

 0-InputLayer                                                         (None, 3, 8, 8)
 1-Conv2DLayer              3x3 ::2 ::2               elu             (None, 3, 5, 5)
 2-Pool2DLayer              3x3 ::2 ::2               max             (None, 3, 2, 2)
 3-DimshuffleLayer                                                   
 4-ReshapeLayer                                                       (None, 1, 1, 12)



In [21]:
X = np.zeros((1,3,7,7),dtype=np.float32)
X[0,0,0,0] = 1
#X[0,0,2,3] = 1
X[0,0,5,6] = 1
print X

Y = lasagne.layers.get_output(layers,X)

[[[[ 1.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  1.]
   [ 0.  0.  0.  0.  0.  0.  0.]]

  [[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]

  [[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]


In [22]:
#print(Y.shape)
print Y
for out in Y:
    print out.eval()[0]
res = Y[4].eval()

[TensorConstant{[[[[ 1.  0..0.  0.]]]]}, Elemwise{switch,no_inplace}.0, Pool{ignore_border=False, mode='max', ndim=2}.0, InplaceDimShuffle{0,1,3,2}.0, Reshape{4}.0]
[[[ 1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  1.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.]]]
[[[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  0.  0.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1. 

In [23]:
l_deconv = receptive_field_build_deconv_layers(l_out,layers[1])
layers_deconv = lasagne.layers.get_all_layers(l_deconv)
print_layers(layers_deconv)

 0-InputLayer                                                         (None, 1, 1, 12)
 1-ReshapeLayer                                                       (None, 3, 2, 2)
 2-DimshuffleLayer                                                   
 3-ReversedPool2DLayer      3x3 ::2 ::2               rectify         (None, 3, 5, 5)
 4-ReversedConv2DLayer      3x3 ::2 ::2               rectify         (None, 3, 8, 8)



In [24]:
Units_on = res>0
Units_on = Units_on.astype(np.float32)
Units_off = res<=0
Units_off = Units_off.astype(np.float32)
print Units_on
print Units_off

[[[[ 1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  1.]]]]
[[[[ 0.  1.  1.  0.  0.  1.  1.  0.  0.  1.  1.  0.]]]]


In [25]:
Y_on = lasagne.layers.get_output(layers_deconv,Units_on)
Y_off = lasagne.layers.get_output(layers_deconv,Units_off)

Shape.0 (2, 2) (0, 0)
Shape.0 (2, 2) (0, 0)


In [26]:
for out_on,out_off in zip(Y_on,Y_off):
    res_on = out_on.eval()[0]
    res_off = out_off.eval()[0]
    res2 = (res_on>0) & (res_off<=0)
    res2 = res2.astype(np.float32)
    print res2
    
print 'res_on',res_on
print 'res_off',res_off

[[[ 1.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  1.]]]
[[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
[[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]
[[[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  1.]
  [ 0.  0.  0.  1.  1.]]]
[[[ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]
  [ 0.  0.  0.  0.  0.  1.  1.  1.]]

 [[ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.

In [17]:
l_in = lasagne.layers.InputLayer((None,1,2,10))
l_dil = DilationSeparate2DLayer(l_in,(1,3),((0,0),(0,2)))
l_mrg = DilationMerge2DLayer(l_dil,(1,3),((0,0),(0,2)))

layers_mrg = lasagne.layers.get_all_layers(l_mrg)
print_layers(layers_mrg)

 0-InputLayer                                                         (None, 1, 2, 10)
 1-DilationSeparate2DLayer                                            (None, 1, 2, 4)
 2-DilationMerge2DLayer                                               (None, 1, 2, 10)



In [18]:
X = np.zeros((2,1,2,10),dtype=np.float32)
X[0,0,0,:] = np.arange(1,11)
X[0,0,1,:] = np.arange(11,21)
X[1,0,0,:] = np.arange(21,31)
X[1,0,1,:] = np.arange(31,41)
print X

Y = lasagne.layers.get_output(l_mrg,X)

[[[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
   [ 11.  12.  13.  14.  15.  16.  17.  18.  19.  20.]]]


 [[[ 21.  22.  23.  24.  25.  26.  27.  28.  29.  30.]
   [ 31.  32.  33.  34.  35.  36.  37.  38.  39.  40.]]]]


In [19]:
print(Y.eval())

[[[[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.]
   [ 11.  12.  13.  14.  15.  16.  17.  18.  19.  20.]]]


 [[[ 21.  22.  23.  24.  25.  26.  27.  28.  29.  30.]
   [ 31.  32.  33.  34.  35.  36.  37.  38.  39.  40.]]]]


In [20]:
#test_dilation.test_stride_reshape_layer()
convvisual.tests.test_dilation.test_inverse_dilation_layer()

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)

 0-InputLayer                                                         (None, 1, 15, 1)
 1-ReversedFinalReshapeLayer                                          (None, 1, 4, 1)
 2-DilationMerge2DLayer                                               (None, 1, 8, 1)
 3-DilationMerge2DLayer                                               (None, 1, 15, 1)

[array([[[[  1.],
         [  2.],
         [  3.],
         [  4.],
         [  5.],
         [  6.],
         [  7.],
         [  8.],
         [  9.],
         [ 10.],
         [ 11.],
         [ 12.],
         [ 13.],
         [ 14.],
         [ 15.]]],


       [[[ 16.],
         [ 17.],
         [ 1

In [21]:
convvisual.tests.test_dilation.test_dilation_layer()

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)

 0-InputLayer                                                         [None, 1, 15, 1]
 1-StrideReshapeLayer           ::2 ::1                               (None, 1, 8, 1)
 2-StrideReshapeLayer           ::2 ::1                               (None, 1, 4, 1)
 3-FinalReshapeLayer                                                  (None, 1, 15, 1)
 4-DilationSeparate2DLayer                                            (None, 1, 8, 1)
 5-DilationSeparate2DLayer                                            (None, 1, 4, 1)
 6-DilationMerge2DLayer                                               (None, 1, 15, 1)

[array([[[[  1.],
         [  2.],
         [  

In [37]:
convvisual.tests.test_inverse_conv.test_inv_conv()

[inputs, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0]
0 [[[[  1.   1.   1.   1.   1.   1.   1.]
   [  1.   2.   1.   1.   1.   1.   1.]
   [  1.   1.   3.   1.   1.   1.   1.]
   [  1.   1.   1.   4.   1.   1.   1.]
   [  1.   1.   1.   1.   5.   1.   1.]
   [  1.   1.   1.   1.   1.   6.   1.]
   [  1.   1.   1.   1.   1.   1.   7.]]]


 [[[  5.   5.   5.   5.   5.   5.   5.]
   [  5.   6.   5.   5.   5.   5.   5.]
   [  5.   5.   7.   5.   5.   5.   5.]
   [  5.   5.   5.   8.   5.   5.   5.]
   [  5.   5.   5.   5.   9.   5.   5.]
   [  5.   5.   5.   5.   5.  10.   5.]
   [  5.   5.   5.   5.   5.   5.  11.]]]]
1 [[[[  2.   3.   2.   2.   2.   2.   2.]
   [  3.   4.   5.   3.   3.   3.   3.]
   [  3.   4.   5.   6.   3.   3.   3.]
   [  3.   3.   5.   6.   7.   3.   3.]
   [  3.   3.   3.   6.   7.   8.   

ValueError: operands could not be broadcast together with shapes (2,1,13,1) (2,1,7,7) 

In [ ]:
np.diag([1,2,3,4,5])+1

In [ ]:
a = (1,2,3,4)
a[2:3]